<a href="https://colab.research.google.com/github/KauaHenSilva/python_neural_networks_for_image_classification/blob/main/python_neural_networks_for_image_classification_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
import requests
import os
import zipfile
import shutil

In [2]:
def download_large_file_from_google_drive(file_id, destination):
    base_url = "https://drive.google.com/uc?export=download"

    with requests.Session() as session:
        response = session.get(base_url, params={'id': file_id}, stream=True)
        for key, value in response.cookies.items():
            if key.startswith('download_warning'):
                confirm_token = value
                break
        else:
            confirm_token = None

        if confirm_token:
            params = {'id': file_id, 'confirm': confirm_token}
            response = session.get(base_url, params=params, stream=True)

        # Escrever o conteúdo do arquivo no destino
        with open(destination, 'wb') as f:
            for chunk in response.iter_content(32768):
                f.write(chunk)

download_large_file_from_google_drive('1j5ZM-csz4Ue6xBJPcztfJ-z7Fc6eu82n', 'homer_bart.zip')

In [3]:
if not os.path.exists('homer_bart'):
    with zipfile.ZipFile('homer_bart.zip', 'r') as zip_ref:
        zip_ref.extractall('homer_bart_temp')

    os.remove('homer_bart.zip')
    shutil.move('homer_bart_temp/homer_bart_1', 'homer_bart')
    shutil.rmtree('homer_bart_temp')

# Pré tratamento

In [4]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

In [5]:
paths_datas = [os.path.join('./homer_bart', x) for x in sorted(os.listdir('homer_bart/'))]
paths_datas = paths_datas[1:]

In [6]:
export = 'boca,calca,sapato,camisa,calcao,tenis,classes\n'
sizeH, sizeW = 224, 224
mostrar_imagem = False
caracteristicas = []

In [7]:
imagens_np = []
classes = []

for path in paths_datas:
        try:
              imagem = cv2.imread(path)
              imagem = cv2.resize(imagem, (sizeH, sizeW))
        except:
            continue

        imagem_alterada = imagem.copy()
        imagem_caracteristicas = []

        boca = 0
        calca = 0
        sapato = 0
        camisa = 0
        calcao = 0
        tenis = 0

        name_img = os.path.basename(path)

        if name_img.startswith('bart'):
            classe = 0
        else:
            classe = 1


        for altura in range(0, sizeH):
            for largura in range(0, sizeW):
                # RGB -> BGR
                azul = imagem_alterada.item(altura, largura, 0)
                verde = imagem_alterada.item(altura, largura, 1)
                vermelho = imagem_alterada.item(altura, largura, 2)

                # Homer - marrom boca
                if (azul >= 95 and azul <= 140) and (verde >= 160 and verde <= 185) and (vermelho >= 175 and vermelho <= 205):
                    imagem_alterada[altura, largura] = [0, 255, 255]
                    boca += 1

                # Homer - azul calca
                if (azul >= 150 and azul <= 180) and (verde >= 98 and verde <= 120) and (vermelho >= 0 and vermelho <= 90):
                    imagem_alterada[altura, largura] = [0, 255, 255]
                    calca += 1

                # Homer - cinza sapato
                if altura > sizeH /2:
                  if (azul >= 25 and azul <= 45) and (verde >= 25 and verde <= 45) and (vermelho >= 25 and vermelho <= 45):
                      imagem_alterada[altura, largura] = [0, 255, 255]
                      sapato += 1

                # Bart - laranja da camisa
                if (azul >= 11 and azul <= 50 and verde >= 85 and verde <= 105 and vermelho >= 240 and vermelho <= 255):
                  imagem_alterada[altura, largura] = [0, 255, 128]
                  camisa += 1

                # Bart - azul do calção
                if (azul >= 125 and azul <= 170 and verde >= 0 and verde <= 12 and vermelho >= 0 and vermelho <= 20):
                  imagem_alterada[altura, largura] = [0, 255, 128]
                  calcao += 1

                # Bart - azul do tênis
                if altura > (sizeH / 2):
                  if (azul >= 125 and azul <= 170 and verde >= 0 and verde <= 12 and vermelho >= 0 and vermelho <= 20):
                    imagem_alterada[altura, largura] = [0, 255, 128]
                    tenis += 1


        boca = round((boca / (sizeH * sizeW)) * 100, 9)
        calca = round((calca / (sizeH * sizeW)) * 100, 9)
        sapato = round((sapato / (sizeH * sizeW)) * 100, 9)
        camisa = round((camisa / (sizeH * sizeW)) * 100, 9)
        calcao = round((calcao / (sizeH * sizeW)) * 100, 9)
        tenis = round((tenis / (sizeH * sizeW)) * 100, 9)

        imagem_caracteristicas.append(boca)
        imagem_caracteristicas.append(calca)
        imagem_caracteristicas.append(sapato)
        imagem_caracteristicas.append(camisa)
        imagem_caracteristicas.append(calcao)
        imagem_caracteristicas.append(tenis)
        imagem_caracteristicas.append(classe)

        caracteristicas.append(imagem_caracteristicas)

        f = (','.join([str(x) for x in imagem_caracteristicas]))
        export += f'{f}\n'

        if mostrar_imagem:
            imagem_alterada = cv2.cvtColor(imagem_alterada, cv2.COLOR_BGR2RGB)
            imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)
            figure, ax = plt.subplots(1, 2)
            ax[0].imshow(imagem)
            ax[1].imshow(imagem_alterada)
            plt.show()

In [8]:
with open('caracteristicas.csv', 'w') as f:
    f.write(export)


In [9]:
pd_caracteristicas = pd.read_csv('caracteristicas.csv')
pd_caracteristicas

,boca,calca,sapato,camisa,calcao,tenis,classes
0,0.000000,0.000000,0.259088,6.698422,3.336256,3.336256,0
1,0.000000,0.000000,0.177376,4.880820,3.061224,3.061224,0
2,0.000000,0.000000,0.181362,5.113999,4.920679,4.920679,0
3,0.000000,0.532127,0.153460,0.000000,0.000000,0.000000,0
4,0.000000,0.000000,0.161432,8.793048,3.380102,3.380102,0
...,...,...,...,...,...,...,...
264,6.357621,0.000000,0.261081,0.000000,0.000000,0.000000,1
265,0.000000,0.000000,0.251116,0.000000,0.000000,0.000000,1
266,4.069675,0.000000,0.581952,0.000000,0.000000,0.000000,1
267,1.379145,0.000000,0.089684,0.000000,0.000000,0.000000,1
